In [1]:
%matplotlib inline

In [2]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [3]:
!ls /home/mobile/Downloads/SynthText/

1    112  126  14   153  167  180  194	27  40	54  68	81  95
10   113  127  140  154  168  181  195	28  41	55  69	82  96
100  114  128  141  155  169  182  196	29  42	56  7	83  97
101  115  129  142  156  17   183  197	3   43	57  70	84  98
102  116  13   143  157  170  184  198	30  44	58  71	85  99
103  117  130  144  158  171  185  199	31  45	59  72	86  gt.mat
104  118  131  145  159  172  186  2	32  46	6   73	87  image_shape.pkl
105  119  132  146  16	 173  187  20	33  47	60  74	88
106  12   133  147  160  174  188  200	34  48	61  75	89
107  120  134  148  161  175  189  21	35  49	62  76	9
108  121  135  149  162  176  19   22	36  5	63  77	90
109  122  136  15   163  177  190  23	37  50	64  78	91
11   123  137  150  164  178  191  24	38  51	65  79	92
110  124  138  151  165  179  192  25	39  52	66  8	93
111  125  139  152  166  18   193  26	4   53	67  80	94


In [4]:
# read matlab mat format file
import scipy.io as sio

In [5]:
data_dir = '/home/mobile/Downloads/SynthText/'

In [6]:
gt = sio.loadmat(data_dir + 'gt.mat')

In [11]:
num = gt['imnames'].shape[1]

In [13]:
# constant needed in split_to_size
int(num * 0.9), num - int(num * 0.9)

(772875, 85875)

In [ ]:
# the name list of image
idx_of_img = 4
image_file = data_dir + str(gt['imnames'][:,idx_of_img][0][0])

In [ ]:
im  = cv2.cvtColor(cv2.imread(image_file), cv2.COLOR_BGR2RGB)

In [ ]:
# list of words seems useless in this scenario
for i, txt in enumerate(gt['txt'][:,idx_of_img][0]):
    print("{i}th word is {word}".format(i=i, word=txt))

In [ ]:
# recaculate bbox of words
wordBB = gt['wordBB'][:,idx_of_img][0]

In [ ]:
# convert wordBB to aabb boxes list
def convert_rotbbox_to_aabb(wordBB, image_size):
    word_len = wordBB.shape[2]
    height, width = image_size
    aabb = np.zeros((word_len, 4), dtype=np.float32)
    for i in xrange(word_len):
        x_min = np.min(wordBB[0, :, i]) / float(width)
        x_max = np.max(wordBB[0, :, i]) / float(width)
        y_min = np.min(wordBB[1, : ,i]) / float(height)
        y_max = np.max(wordBB[1, : ,i]) / float(height)
        aabb[i,:] = np.array([y_min, x_min, y_max, x_max])
        
    return aabb

In [ ]:
aabb = convert_rotbbox_to_aabb(wordBB, im.shape[:2])

In [ ]:
for bbox in aabb:
    y_min, x_min, y_max, x_max = bbox
    cv2.rectangle(im, 
                  (int(x_min*im.shape[1]), int(y_min*im.shape[0])), 
                  (int(x_max*im.shape[1]), int(y_max*im.shape[0])), 
                  (255, 0, 0), 2)

In [ ]:
plt.figure()
plt.imshow(im)
plt.show()

In [ ]:
# should read all image's size into a single file
from tqdm import tqdm
import pickle

In [ ]:
with open('image_size.pkl', 'w') as f:
    dst = {}
    num_of_file = gt['imnames'].shape[1]
    for idx_of_img in tqdm(xrange(num_of_file)):
        image_file_name = str(gt['imnames'][:,idx_of_img][0][0])
        path = data_dir + image_file_name
        height, width, depth = cv2.imread(path).shape
        dst[image_file_name] = [height, width, depth]
    
    pickle.dump(dst, f, pickle.HIGHEST_PROTOCOL)

# to tf-record

In [ ]:
import os
import tensorflow as tf

In [ ]:
VOC_LABELS = {
    'none': (0, 'Background'),
    'text': (1, 'text')
}

In [ ]:
with open('image_shape.pkl') as f:
    dst = pickle.load(f)

In [ ]:
import sys

In [ ]:
sys.path.append('../datasets/')

In [ ]:
from dataset_utils import int64_feature, float_feature, bytes_feature

In [ ]:
import random
import logging

In [ ]:
logger = logging.getLogger()

In [ ]:
logger.setLevel(logging.DEBUG)

In [ ]:
SAMPLES_PER_FILES = 200

In [ ]:
# convert wordBB to aabb boxes list
def convert_rotbbox_to_aabb(wordBB, image_size):
    if len(wordBB.shape) < 3:
        wordBB = np.expand_dims(wordBB, axis=-1)

    word_len = wordBB.shape[2]
    height, width = image_size
    aabb = np.zeros((word_len, 4), dtype=np.float32)
    for i in xrange(word_len):
        x_min = np.min(wordBB[0, :, i]) / float(width)
        x_max = np.max(wordBB[0, :, i]) / float(width)
        y_min = np.min(wordBB[1, : ,i]) / float(height)
        y_max = np.max(wordBB[1, : ,i]) / float(height)
        aabb[i,:] = np.array([y_min, x_min, y_max, x_max])

    return aabb

def run(dataset_dir, output_dir, split_ratio=0.9, shuffling=False):
    """Runs the conversion operation.

    Args:
      dataset_dir: The dataset directory where the dataset is stored.
      output_dir: Output directory.
    """
    
    def _process_image(directory, idx_of_img):
        """Process a image and annotation file.

        Args:
          directory: the syntext data directory
          idx_of_img: int, the index of images in gt.mat
        Returns:
          image_data: string, JPEG encoding of RGB image.
          shape: the image shape
          bboxes: the bounding box of text
          labels: the label of text always 1, there is only one label
        """
        # Read the image file.
        name = str(gt['imnames'][:,idx_of_img][0][0])
        filename = directory + name
        image_data = tf.gfile.FastGFile(filename, 'r').read()

        shape = image_shapes[name]

        wordBB = gt['wordBB'][:,idx_of_img][0]

        bboxes = convert_rotbbox_to_aabb(wordBB, shape[:2])
        labels = [VOC_LABELS['text'][0]] * len(bboxes)

        return image_data, shape, bboxes, labels
    
    def _convert_to_example(image_data, shape, bboxes, labels):
        """Build an Example proto for an image example.

        Args:
          image_data: string, JPEG encoding of RGB image; Sythtext images are always .jpg
          labels: list of integers, identifier for the ground truth;
          bboxes: list of bounding boxes; each box is a list of integers;
              specifying [xmin, ymin, xmax, ymax]. All boxes are assumed to belong
              to the same label as the image label.
          shape: 3 integers, image shapes in pixels.
        Returns:
          Example proto
        """
        xmin = []
        ymin = []
        xmax = []
        ymax = []
        for b in bboxes:
            assert len(b) == 4
            # pylint: disable=expression-not-assigned
            [l.append(point) for l, point in zip([ymin, xmin, ymax, xmax], b)]
            # pylint: enable=expression-not-assigned

        image_format = b'JPEG'
        example = tf.train.Example(features=tf.train.Features(feature={
                'image/height': int64_feature(shape[0]),
                'image/width': int64_feature(shape[1]),
                'image/channels': int64_feature(shape[2]),
                'image/shape': int64_feature(shape),
                'image/object/bbox/xmin': float_feature(xmin),
                'image/object/bbox/xmax': float_feature(xmax),
                'image/object/bbox/ymin': float_feature(ymin),
                'image/object/bbox/ymax': float_feature(ymax),
                'image/object/bbox/label': int64_feature(labels),
                'image/format': bytes_feature(image_format),
                'image/encoded': bytes_feature(image_data)}))
        return example
    
    def _add_to_tfrecord(dataset_dir, idx, tfrecord_writer):
        """Loads data from image and annotations files and add them to a TFRecord.

        Args:
          dataset_dir: Dataset directory;
          name: Image name to add to the TFRecord;
          tfrecord_writer: The TFRecord writer to use for writing.
        """
        image_data, shape, bboxes, labels = \
            _process_image(dataset_dir, idx)
        example = _convert_to_example(image_data, shape, bboxes, labels)
        tfrecord_writer.write(example.SerializeToString())
        
    def _get_output_filename(output_dir, split_name, idx):
        return '%s/syntext_%s_%04d.tfrecord' % (output_dir, split_name, idx)

    if not tf.gfile.Exists(dataset_dir):
        tf.gfile.MakeDirs(dataset_dir)

    print('read gt.mat file...')
    gt = sio.loadmat(data_dir + 'gt.mat')
    
    print('read image_shape.pkl file...')
    with open('image_shape.pkl') as f:
        image_shapes = pickle.load(f)
    
    num_of_image = gt['imnames'].shape[1]
    
    assert num_of_image == len(image_shapes)
    
    # assert split_ratio belong to (0, 1)
    num_for_train = int(num_of_image * split_ratio)
    
    # shuffle the indices
    fileidxs = range(0, num_of_image)
    if shuffling:
        random.seed(RANDOM_SEED)
        random.shuffle(fileidxs)
    
    # pick up indices for train and test
#     idxs_for_train = fileidxs[:num_for_train]
#     idxs_for_test = fileidxs[num_for_train:]
    idxs_for_train = fileidxs[:200]
    idxs_for_test = fileidxs[200:500]

    # Process dataset files for trains
    logging.info('convert data for train.')

    fidx = 0
    for idx in tqdm(idxs_for_train):
        if idx % SAMPLES_PER_FILES == 0:
            # Open new TFRecord file.
            tf_filename = _get_output_filename(output_dir, 'train', fidx)
            fidx += 1
            
        with tf.python_io.TFRecordWriter(tf_filename) as tfrecord_writer:
            _add_to_tfrecord(dataset_dir, idx, tfrecord_writer)
    
    # Process dataset files for test
    logging.info('convert data for test')

    fidx = 0
    for idx in tqdm(idxs_for_test):
        if idx % SAMPLES_PER_FILES == 0:
            # Open new TFRecord file.
            tf_filename = _get_output_filename(output_dir, 'test', fidx)
            fidx += 1
            
        with tf.python_io.TFRecordWriter(tf_filename) as tfrecord_writer:
            _add_to_tfrecord(dataset_dir, idx, tfrecord_writer)
    
    print('\nFinished converting the Syntext dataset!')

In [ ]:
run(data_dir, '/tmp/')